In [1]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [2]:
engine = create_engine("sqlite:///../db/apache.sqlite")
base= automap_base()
base.prepare(engine, reflect=True)
inspector = inspect(engine)
log_table = base.classes.access_response
session = Session(engine)

In [3]:
#spamhaus droplist
drop_list = pd.read_csv('/home/spinster/bootcamp/homework/apache/blacklist/drop.txt', delimiter='\ ')

#spamhaus extended droplist
edrop_list = pd.read_csv('/home/spinster/bootcamp/homework/apache/blacklist/edrop.txt', delimiter='\ ')

#https://myip.ms/ blacklist as of 06/12/18
my_ip_ms_list = pd.read_csv('/home/spinster/bootcamp/homework/apache/blacklist/full_blacklist_database.csv', delimiter=',')

#zeus list
zeus_list = pd.read_csv('/home/spinster/bootcamp/homework/apache/blacklist/zeus.csv', delimiter = ',')

#malware.com domain list
malware_list = pd.read_csv('/home/spinster/bootcamp/homework/apache/blacklist/malware_domain_list.csv', delimiter = ',')

/home/spinster/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/spinster/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [4]:
# drop weird stuff
drop_list_df = drop_list[[';']]
edrop_list_df = edrop_list[[';']]

In [5]:
drop_list_df.head()

,;
0,;
1,;
2,;
3,1.10.16.0/20
4,1.32.128.0/18


In [6]:
# drop more weird stuff
drop_list_df = drop_list_df.iloc[3:]
edrop_list_df = edrop_list_df.iloc[3:]

In [7]:
# drop port numbers
drop_list_df = drop_list_df[';'].map(lambda x: x[:-3])
edrop_list_df = edrop_list_df[';'].map(lambda x: x[:-3])

In [8]:
#analysis done on each list, then added all together. Nothing was on the spamhaus lists
blacklist = list(drop_list_df) + list(edrop_list_df) + list(my_ip_ms_list['ip']) + list(zeus_list['ip']) + list(malware_list['ip'])

In [9]:
zeus_list.head()

,ip
0,101.200.81.187
1,103.19.89.118
2,103.230.84.239
3,103.4.52.150
4,103.7.59.135


In [10]:
len(blacklist)

4842

In [11]:
# dataframe from which to do analysis
blacklist_df = pd.DataFrame({'ip':blacklist})

In [12]:
# match? yes = 1
blacklist_df['True']=1

In [13]:
blacklist_df.head()

,ip,True
0,1.10.16.0,1
1,1.32.128.0,1
2,5.8.37.0,1
3,5.34.242.0,1
4,5.101.218.0,1


In [14]:
# create query & df to use for query
ip_query = 'select * from access_response'
ip_blacklist = pd.read_sql_query(ip_query, session.bind)

In [15]:
ip_blacklist

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,1,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,-,220,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
1,2,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/PMA/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
2,3,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/dbadmin/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
3,4,62.170.2.205,2018-04-08 07:19:26,GMT-0700,HEAD,http://198.27.242.168:80/pma/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
4,5,62.170.2.205,2018-04-08 07:19:26,GMT-0700,HEAD,http://198.27.242.168:80/db/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
5,6,108.41.39.187,2018-04-08 07:53:03,GMT-0700,GET,/phone_pix/cat.jpg,404,-,534,just some guy,40.5833,-73.9240
6,7,74.208.182.37,2018-04-08 09:00:47,GMT-0700,GET,/,200,-,562,Mozilla/4.0 (compatible; MSIE 5.0; Windows NT;...,40.0548,-75.4083
7,8,46.229.164.102,2018-04-08 09:58:37,GMT-0700,GET,/robots.txt,200,-,385,Mozilla/5.0 (compatible; SemrushBot/1.2~bl; +h...,39.0180,-77.5390
8,9,96.57.208.218,2018-04-08 09:59:37,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,-,220,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.7510,-97.8220
9,10,96.57.208.218,2018-04-08 09:59:37,GMT-0700,HEAD,http://198.27.242.168:80/PMA/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.7510,-97.8220


In [16]:
# merge the blacklist with the df of database to compare
blacklist_hits_df = pd.merge(blacklist_df, ip_blacklist[['ip','DateTime','URL', 'latitude', 'longitude']], how='right', on='ip')

In [17]:
# drop NaN, which are the rows that did not match the blacklists
blacklist_hits_df = blacklist_hits_df.dropna()

In [18]:
# hits from blacklisted ips
blacklist_hits_df

,ip,True,DateTime,URL,latitude,longitude
0,23.103.64.39,1.0,2018-03-11 23:57:33,/ramble/index.php?blogId=83,47.6801,-122.1206
1,62.210.178.179,1.0,2015-11-21 20:41:07,/ramble/index.php?blogId=46,48.8582,2.3387
2,62.210.178.179,1.0,2015-11-21 20:41:07,/,48.8582,2.3387
3,62.210.178.179,1.0,2015-09-30 21:47:17,/ramble/index.php?blogId=70,48.8582,2.3387
4,62.210.178.179,1.0,2015-09-30 21:47:17,/,48.8582,2.3387
5,101.255.17.34,1.0,2015-12-11 05:32:43,/ramble/index.php?blogId=54,-6.1744,106.8294
6,101.255.17.34,1.0,2015-12-11 05:32:43,/,-6.1744,106.8294
7,103.27.236.246,1.0,2015-12-02 06:52:42,/ramble/index.php?blogId=54,10.8142,106.6438
8,103.27.236.246,1.0,2015-12-02 06:52:42,/,10.8142,106.6438
9,103.27.236.246,1.0,2015-11-26 09:42:25,/ramble/index.php?blogId=54,10.8142,106.6438


In [19]:
blacklist_hits_df.to_csv('../db/blacklist_hits.csv', sep=',')

In [20]:
len(blacklist_hits_df)

29

In [21]:
# uniques from blacklisted ips
unique_ips_from_blacklist = blacklist_hits_df['ip'].unique()

In [22]:
len(unique_ips_from_blacklist)

11

# Ignore the rest. This analysis is not that interesting

In [23]:
# create query to find multiple hits at the same time
ip_query2 = 'select ip, DateTime, URL from access_response'

hits_by_dt_df = pd.read_sql_query(ip_query2, session.bind)

In [24]:
hits_by_dt_df.head()

,ip,DateTime,URL
0,62.170.2.205,2018-04-08 07:19:25,http://198.27.242.168:80/phpmyadmin/
1,62.170.2.205,2018-04-08 07:19:25,http://198.27.242.168:80/PMA/
2,62.170.2.205,2018-04-08 07:19:25,http://198.27.242.168:80/dbadmin/
3,62.170.2.205,2018-04-08 07:19:26,http://198.27.242.168:80/pma/
4,62.170.2.205,2018-04-08 07:19:26,http://198.27.242.168:80/db/


In [25]:
# figure out datetime format
x = '2018-04-08 07:19:25'
dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

datetime.datetime(2018, 4, 8, 7, 19, 25)

In [26]:
# convert datetime
hits_by_dt_df['DateTime'] = hits_by_dt_df['DateTime'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [27]:
#separate DateTime into two columns
hits_by_dt_df['Date'] = hits_by_dt_df['DateTime'].map(lambda x: dt.datetime.date(x))
hits_by_dt_df['Time'] = hits_by_dt_df['DateTime'].map(lambda x: dt.datetime.time(x))

In [28]:
# drop the DateTime column
hits_by_dt_df.drop(['DateTime'], axis=1, inplace=True)

In [29]:
# counting hits in a row
in_a_row = hits_by_dt_df.groupby(['ip','Date','Time']).count()

In [30]:
# how many hits did the same URL get?
in_a_row_blast = in_a_row.sort_values('URL', ascending=False)

In [42]:
# this first IP appears to be mine, which is odd, since I removed my LAN IP
in_a_row_blast.head(25)

URL
ip              Date       Time         
172.92.10.209   2018-02-20 22:02:10  111
                           21:55:19   72
                           22:03:06   60
                           21:58:26   56
                           21:58:59   53
                           21:58:03   47
                           21:58:11   46
                           22:23:43   42
                           21:58:27   42
                           21:55:18   38
                           22:02:20   34
104.217.216.170 2015-10-01 14:46:35   32
                           14:46:36   31
108.64.229.11   2018-03-11 20:47:14   31
104.217.216.170 2015-09-29 04:19:03   27
107.183.220.254 2015-04-24 13:48:52   26
104.217.216.170 2015-09-29 04:19:01   22
                           04:19:02   19
66.249.84.13    2018-03-07 22:07:59   18
66.249.84.15    2018-03-07 22:07:59   18
104.217.216.170 2015-10-01 14:46:37   17
23.239.208.54   2015-07-09 23:24:47   16
148.163.2.234   2015-07-22 08:29:15   16
23.239.208.33   2015-06-29 08:38:27   15
172.92.10.209   2018-02-20 21:58:43   14

In [32]:
ip_query3 = 'select ip, DateTime, URL, Method from access_response'

In [33]:
hits_by_method_df = pd.read_sql_query(ip_query3, session.bind)

In [38]:
method_count = hits_by_method_df.groupby(['Method']).count()

In [39]:
method_count

,ip,DateTime,URL
Method,,,
CONNECT,51,51,51
DELETE,1,1,1
GET,28608,28608,28608
HEAD,1715,1715,1715
OPTION,6,6,6
OPTIONS,51,51,51
POST,233,233,233
PROPFIND,21,21,21
PUT,6,6,6
